In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from tqdm import tqdm
import matplotlib.pyplot as plt

# Baseline Models

With this notebook we want to create a compareable baseline for our other models.

Therefore we use the strategies "most_frequent" and "prior". With those we want to evaluate on how well a model would predict if it always predicts the most common rating aswell as predict exactly the distribution of the training set.

In [5]:
def split_X_y(df):
    return df[["UserID", "WineID", "Vintage", "Date"]], df["Rating"]

In [6]:
df_train = pd.read_csv(r"../trainset.csv")
df_train_X, df_train_y = split_X_y(df_train)

In [3]:
df_hot_user_hot_wine = pd.read_csv(r"../testset_warm_user_warm_item.csv")
df_hot_user_cold_wine = pd.read_csv(r"../testset_warm_user_cold_item.csv")
df_cold_user_hot_wine = pd.read_csv(r"../testset_cold_user_warm_item.csv")
df_cold_user_cold_wine = pd.read_csv(r"../testset_cold_user_cold_item.csv")

In [7]:
df_hot_user_hot_wine_X, df_hot_user_hot_wine_y = split_X_y(df_hot_user_hot_wine)
df_hot_user_cold_wine_X, df_hot_user_cold_wine_y = split_X_y(df_hot_user_cold_wine)
df_cold_user_hot_wine_X, df_cold_user_hot_wine_y = split_X_y(df_cold_user_hot_wine)
df_cold_user_cold_wine_X, df_cold_user_cold_wine_y = split_X_y(df_cold_user_cold_wine)

In [8]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train_X, df_train_y)

DummyClassifier(strategy='most_frequent')

In [9]:
def bootstrap_mae(y_true, y_pred, n_bootstrap=1000, ci=95, random_state=None):
    rng = np.random.default_rng(random_state)
    mae_scores = []

    n = len(y_true)
    for _ in tqdm(range(n_bootstrap)):
        # Sample indices with replacement
        indices = rng.choice(n, size=n, replace=True)
        sample_true = y_true[indices]
        sample_pred = y_pred[indices]
        
        mae = mean_absolute_error(sample_true, sample_pred)
        mae_scores.append(mae)
        
    mae_scores = np.array(mae_scores)

    # Confidence interval
    lower = np.percentile(mae_scores, (100 - ci) / 2)
    upper = np.percentile(mae_scores, 100 - (100 - ci) / 2)

    return {
        'mae_distribution': mae_scores,
        'mean': mae_scores.mean(),
        'ci': (lower, upper)
    }

### Evaluation

In [12]:
df_hot_user_hot_wine_pred = dummy_clf.predict(df_hot_user_hot_wine_X)
df_hot_user_cold_wine_pred = dummy_clf.predict(df_hot_user_cold_wine_X)
df_cold_user_hot_wine_pred = dummy_clf.predict(df_cold_user_hot_wine_X)
df_cold_user_cold_wine_pred = dummy_clf.predict(df_cold_user_cold_wine_X)

In [16]:
res_hot_user_hot_wine = bootstrap_mae(df_hot_user_hot_wine_pred, df_hot_user_hot_wine_y, n_bootstrap=100, ci=95)
print(f"MAE hot_user_hot_wine: {res_hot_user_hot_wine['mean']:.4f}")
print(f"95% CI: [{res_hot_user_hot_wine['ci'][0]:.4f}, {res_hot_user_hot_wine['ci'][1]:.4f}]")

res_hot_user_cold_wine = bootstrap_mae(df_hot_user_cold_wine_pred, df_hot_user_cold_wine_y, n_bootstrap=100, ci=95)
print(f"MAE hot_user_cold_wine: {res_hot_user_cold_wine['mean']:.4f}")
print(f"95% CI: [{res_hot_user_cold_wine['ci'][0]:.4f}, {res_hot_user_cold_wine['ci'][1]:.4f}]")

res_cold_user_hot_wine = bootstrap_mae(df_cold_user_hot_wine_pred, df_cold_user_hot_wine_y, n_bootstrap=100, ci=95)
print(f"MAE cold_user_hot_wine: {res_cold_user_hot_wine['mean']:.4f}")
print(f"95% CI: [{res_cold_user_hot_wine['ci'][0]:.4f}, {res_cold_user_hot_wine['ci'][1]:.4f}]")

res_cold_user_cold_wine = bootstrap_mae(df_cold_user_cold_wine_pred, df_cold_user_cold_wine_y, n_bootstrap=100, ci=95)
print(f"MAE cold_user_cold_wine: {res_cold_user_cold_wine['mean']:.4f}")
print(f"95% CI: [{res_cold_user_cold_wine['ci'][0]:.4f}, {res_cold_user_cold_wine['ci'][1]:.4f}]")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.09it/s]


MAE hot_user_hot_wine: 0.4593
95% CI: [0.4587, 0.4598]


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 290.39it/s]


MAE hot_user_cold_wine: 0.4642
95% CI: [0.4590, 0.4691]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.76it/s]


MAE cold_user_hot_wine: 0.5151
95% CI: [0.5138, 0.5165]


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 552.75it/s]

MAE cold_user_cold_wine: 0.6050
95% CI: [0.5963, 0.6146]
